# Notebook for developing and testin the Itinerary Code Pilot

In [ ]:
import requests
import json
import os
import openai
import ls_nb_utils
import pandas as pd
from geopy import distance
import certifi
import logging
from http.client import HTTPConnection
from importlib import reload


In [ ]:
reload(ls_nb_utils)

In [ ]:
openai.api_key  = os.environ['OPENAI_API_KEY']
client = openai.OpenAI()


In [ ]:
ls_app_url =  'https://pocketverse.herokuapp.com/LS_API'
ls_app_username = os.environ['LS_USERNAME']
ls_app_password = os.environ['LS_PASSWORD']
ls_app_username, ls_app_password

In [ ]:
resp_app = ls_nb_utils.get_access_token(ls_app_username, ls_app_password, 'jwt_login', ls_app_url)
resp_app

In [ ]:
token=resp_app['token']

In [ ]:
tok_decoded = ls_nb_utils.decode_jwt(token)
tok_decoded

In [ ]:
uid = tok_decoded['payload']['userId']
uid

In [ ]:
%%time
params = {"searchScope": "FRIEND"}
params = {}
params = {"serachScope": "San Francisco"}
res = ls_nb_utils.test_ls_post(params, 'place/search', url = ls_app_url, token = token, target = 'packet')

In [ ]:
# HTTPConnection.debuglevel = 1
# logging.basicConfig() # you need to initialize logging, otherwise you will not see anything from requests
# logging.getLogger().setLevel(logging.DEBUG)
# requests_log = logging.getLogger("urllib3")
# requests_log.setLevel(logging.DEBUG)
# requests_log.propagate = True

In [ ]:
username = 'mishabello'

In [ ]:
prof_params = {"userName":username}
route = 'lsagent/user/preferences'
prof_res = ls_nb_utils.test_ls_get(route, ls_app_url, params = prof_params, token = token)

In [ ]:
prof_res.content

In [ ]:
uid

In [ ]:
prof_params = {"userId":uid}
route = 'lsagent/user/preferences'
prof_res = ls_nb_utils.test_ls_get(route, ls_app_url, params = prof_params, token = token)

In [ ]:
prefs = prof_res.json()['preferences']
prefs

In [ ]:
prefs_txt = f'{prefs}'
prefs_txt

In [ ]:
places = res['foundPlaces']
len(places)

In [ ]:
places[1]['place']

In [ ]:
places[1]

In [ ]:
places[1]['userStories'][0]

In [ ]:
def make_stories(pl):
    user_stories = pl['userStories']
    location =  pl['place']['locationString']
    user_st_text = f'A number of tourists visited {location} and left their stories.\n'
    for us in user_stories:
        user = us['user']['username']
        story = ''
        for st in us['stories']:
            t = st['timestamp']
            s = st['story']
            if t:
                story += f' on {t} '
            story += s                      
        user_st_text += f'Visitor {user} told the following stories: {story}' + '\n-------------------\n'
    return user_st_text
s = make_stories(places[0])
print(s)

In [ ]:
def select_places(places, value, by = 'user', logic = 'eq' ):
    ret = []
    match by:
        case 'user':
            for pl in places:
                user_stories = pl['userStories']
                for us in user_stories:
                    user = us['user']['username']
                    match logic:
                        case 'eq':
                            if user == value:
                                ret.append(pl)
                        case 'neq':
                            if user != value:
                                ret.append(pl)
                                
        case 'distance':
            orig_lat, orig_long = value['origin']
            orig = (orig_lat, orig_long)
            rad = value['radius_miles']
            for pl in places:
                coor = pl['place']['coordinates']
                lat = coor['lat']
                lng = coor['lng']
                pl_coor = (lat, lng)
                dist = distance.distance(orig, pl_coor).miles
                if (dist <= rad):
                    ret.append(pl)
                
        case _:
            ret = []
    return ret

sel_places = select_places(places, 'mishabello', by = 'user', logic = 'eq')    

In [ ]:
sel_places[0]

In [ ]:
uid == sel_places[0]['userStories'][0]['user']['_id']

In [ ]:
len(sel_places), len(places)

In [ ]:
orig_value = {'origin':(37.7548329217169, -122.44777880753426), 'radius_miles':30}
sel_dist = select_places(places, orig_value, by = 'distance', logic = 'le')
len(sel_dist)

In [ ]:
def make_all_stories(places):
    all_stories = ''
    for pl in places:
        all_stories += make_stories(pl)
    return all_stories

user_stories = make_all_stories(sel_dist)
print(user_stories)

## Using Open AI

In [ ]:
# %pip install geopy

In [ ]:
model_name = "gpt-3.5-turbo-1106"
model_name = "gpt-3.5-turbo"
model_name = "gpt-4"
model_name = "gpt-4-1106-preview"
def req_ai0(system_message, report_message, content):
    # app.logger.debug(system_message)
    messages=[
            {"role": "system", "content": system_message},
            {"role": "system", "content": report_message}, 
            {"role": "user", "content": content}
            ]
    response = client.chat.completions.create(
            model=model_name,
            temperature = 0,
            seed = 1234,
            messages=messages
            )
    answer = response.choices[0].message.content    
    # app.logger.debug(answer)
    try:
        ans = json.loads(answer)
    except:
        ans = {'text':answer}
        
    return ans


In [ ]:
def req_ai(system_message, report_message, content):
    # app.logger.debug(system_message)
    messages=[
            {"role": "system", "content": system_message},
            {"role": "user", "content": content}
            ]
    response = client.chat.completions.create(
            model=model_name,
            temperature = 0,
            seed = 1234,
            messages=messages
            )
    answer = response.choices[0].message.content  


    messages=[
            {"role": "system", "content": report_message}, 
            {"role": "user", "content": answer}]
    
    response = client.chat.completions.create(
            model=model_name,
            temperature = 0,
            seed = 1234,
            messages=messages
            )
    jobject = response.choices[0].message.content     
    # app.logger.debug(answer)
    try:
        ret = json.loads(jobject)
    except:
        ret = {'text':answer}
        
    return ret, answer


In [ ]:
system_message = '''You are a travel agent and a scheduler.
Prepare an itenerary and a schedule based on the user preferences and user stories. 
Reference visitors whose stories we use for the itinerary by name at the end of the response.
'''
report_message = 'Restructure the user content as a JSON object.'

In [ ]:
prefs_txt

In [ ]:
place = 'San Francisco'

date = '2024:04:01'
activities = "golf, lunch, dinner, lodging"
user_stories1 = ''
content = f'''
I am going to be in {place} on {date} and want to do the following acitivities during this day: {activities}.
My preferenses are {prefs_txt}
User stories are: {user_stories}
'''

In [ ]:
content

In [ ]:
res, answer = req_ai(system_message, report_message, content)

In [ ]:
import pprint

In [ ]:
pprint.pprint(res)

In [ ]:
print (answer)

In [ ]:
def structure_response(content):
    model_name = "gpt-3.5-turbo"
    system_message = ''' Structure text provided by user as a json object. Do not loose and of the information provided in the 
                        user content'''
    messages=[
            {"role": "system", "content": system_message},
            {"role": "user", "content": content}
            ]
    response = client.chat.completions.create(
            model=model_name,
            temperature = 0,
            seed = 1234,
            messages=messages
            )
    answer = response.choices[0].message.content  

    jobject = response.choices[0].message.content     
    # app.logger.debug(answer)
    try:
        ret = json.loads(jobject)
    except:
        ret = {'text':answer}      
    return ret, answer   

In [ ]:
r,a = structure_response(answer)

In [ ]:
r

In [ ]:
messages=[
        {"role": "system", "content": 
         '''You compare user stories and the itenerary built on the basis of the user stories and list user stories 
            which were used to buld this itinerary'''},
        {"role": "user", "content": "user stories: " + user_stories}, 
        {"role": "user", "content": "itinerary: " + answer}
        ]
response = client.chat.completions.create(
        model=model_name,
        temperature = 0,
        seed = 1234,
        messages=messages
        )


In [ ]:
print(response.choices[0].message.content )  